# Control Plane: Connect Hosted Agent using Hosting Adapter
----

This notebook summarizes—through an **Azure AI Foundry Control Plane** lens—how to take a locally built agent and use the **Hosting Adapter** to expose it  with the **Foundry SDK (2.0.0b2 preview)** as a **Foundry-compatible HTTP service** with minimal effort.

## What is the Hosting Adapter?
The Hosting Adapter is an abstraction layer that **automatically converts popular agent frameworks (e.g., LangGraph)** into **Microsoft Foundry protocol-compliant HTTP services**.  
It removes the need to manually build REST endpoints, message handling, or streaming logic.

## Key benefits
- **One-line local hosting**: e.g., `from_langgraph(my_agent).run()` → instantly serves a Foundry-compatible service on `localhost:8088` (endpoints + streaming included)
- **Automatic protocol translation**:
    - Conversation management
    - Message serialization/deserialization
    - Foundry-style streaming event generation
- **Built-in production essentials**:
    - OpenTelemetry tracing
    - CORS support
    - SSE (Server-Sent Events) streaming
    - Structured logging
- **Seamless Foundry integration**: Works directly with Foundry **Responses API**, conversation flows, and authentication patterns—simplifying the path from local development to Azure production.

## What this notebook covers
- High-level flow to **host an agent as a Foundry-compatible service** using the Hosting Adapter
- Basic **Control Plane-style checks** (agent/resource visibility and trace verification)

## Table of contents

- [Control Plane overview](#control-plane-overview)
- [Portal vs SDK](#portal-vs-sdk)
- [Setup](#setup)
- [Case 1: Fleet overview](#case-1-fleet-overview)
- [Case 2: Assets](#case-2-assets)
- [Case 3: Quota](#case-3-quota)
- [Additional resources](#additional-resources)
- [Wrap-up](#wrap-up)

## Control Plane overview

### What is the Control Plane?

The Control Plane is the centralized operational surface in Azure AI Foundry for monitoring and governing AI resources across projects.

```
Control Plane = monitoring + management + security + governance
```

### Key areas



## Portal vs SDK
**Recommended usage:**
- Portal: real-time monitoring, compliance/security checks, quota management
- SDK: automation, inventory checks, CI/CD validations

| Feature | Portal | SDK (Python) |
|-----|--------|-------------|
| Fleet overview | ✅ dashboards | ⚠️ limited listing |
| Assets view | ✅ visual | ✅ programmatic |
| Compliance dashboards | ✅ real-time | ❌ not supported |
| Quota management | ✅ manage in Portal | ❌ often read-only/limited |
| Admin / RBAC | ✅ manage access | ❌ not supported |

## MS Foundry Control Plane Accessibility Across Agent Platforms

| Feature        | Foundry Agent | MS Agent Framework | Hosted Agent |
|---------------|---------------|--------------------|--------------|
| Fleet overview | ✅ Yes | ⚠️ with Agent Wrapper  | ✅ Yes |
| Asset view    | ✅ Yes | ⚠️ with Agent Wrapper  | ✅ Yes |

**MAF onboard to MS Foundry Control Plane:**
- MAF + Foundry Agent Wrapper (AzureAIClient)
- MAF → LangGraph → Container → Hosted Agent

**Hosted Agent limitation (2026-01-07):**
- **Region availability:** Hosted Agents are currently available only in **North Central US**.
- **Cross-region deployment:** If you try to deploy Hosted Agents in another region, you may see: *“Hosted Agents are not enabled in this region”* (**HTTP 400 BadRequest**).
- **Network isolation:** Hosted Agents currently can’t be created using the standard setup in **network-isolated** Foundry resources.
- **ACR requirement:** Hosted Agents can be deployed only when **Azure Container Registry (ACR)** is in the **same resource group** and the **same region (North Central US)**.
- **Last verified:** 2026-01-07

## MS Foundry Tools Integration Support Across Agent Platforms 
| Tool | Foundry Agent | MS Agent Framework | Hosted Agent |
|-----|--------|-------------|-------------|
| bing grounding | ✅ Yes | ✅ Yes | ✅ Yes |
| code interpreter | ✅ Yes | ✅ Yes | ✅ Yes |
| browser automation | ✅ Yes (Preview) | ✅ Yes (Preview) | ❌ No |
| computer use | ✅ Yes (Preview) | ✅ Yes (Preview) | ❌ No |
| fabric (Microsoft Fabric) | ✅ Yes (Preview) | ✅ Yes | ❌ No |
| file search | ✅ Yes | ✅ Yes (RAG / AI Search) | ❌ No (Not Built-in) |
| custom function | ✅ Yes | ✅ Yes | ✅ Yes |
| image generation | ✅ Yes (Preview) | ✅ Yes (Preview) | ❌ No |
| mcp (Model Context Protocol) | ✅ Yes | ✅ Yes | ✅ Yes |


## Setup

This notebook reuses the configuration file (`.foundry_config.json`) created by `0_setup/1_setup.ipynb`.

- If the file is missing, run the setup notebook first.
- Make sure you can authenticate (e.g., `az login`), so `DefaultAzureCredential` can work.

In [39]:
# Ensure the notebook kernel can find Azure CLI (`az`) on PATH
import json
import os
import subprocess
from dotenv import load_dotenv
from azure.ai.projects import AIProjectClient
from azure.ai.projects.models import ImageBasedHostedAgentDefinition, ProtocolVersionRecord, AgentProtocol
from azure.identity import DefaultAzureCredential

load_dotenv(override=True)

possible_paths = [
    '/opt/homebrew/bin',  # macOS (Apple Silicon)
    '/usr/local/bin',     # macOS (Intel) / Linux
    '/usr/bin',           # Linux / Codespaces
    '/home/linuxbrew/.linuxbrew/bin',  # Linux Homebrew
]

az_path = None
try:
    result = subprocess.run(['which', 'az'], capture_output=True, text=True)
    if result.returncode == 0:
        az_path = os.path.dirname(result.stdout.strip())
        print(f'🔍 Azure CLI found: {result.stdout.strip()}')
except Exception:
    pass

paths_to_add: list[str] = []
if az_path and az_path not in os.environ.get('PATH', ''):
    paths_to_add.append(az_path)
else:
    for path in possible_paths:
        if os.path.exists(path) and path not in os.environ.get('PATH', ''):
            paths_to_add.append(path)

if paths_to_add:
    os.environ['PATH'] = ':'.join(paths_to_add) + ':' + os.environ.get('PATH', '')
    print(f"✅ Added to PATH: {', '.join(paths_to_add)}")
else:
    print('✅ PATH looks good already')

print(f"\nPATH (first 150 chars): {os.environ['PATH'][:150]}...")

🔍 Azure CLI found: /anaconda/envs/azureml_py38/bin//az
✅ PATH looks good already

PATH (first 150 chars): /anaconda/envs/azureml_py38/bin/:/afh/code/agent-operator-lab/.venv/bin:/home/azureuser/.vscode-server/cli/servers/Stable-994fd12f8d3a5aa16f17d42c041e...


In [3]:
# Load Foundry project settings from .foundry_config.json

import os

config_file = '../0_setup/.foundry_config.json'
try:
    with open(config_file, 'r', encoding='utf-8') as f:
        config = json.load(f)
except FileNotFoundError as e:
    print(f"⚠️ Could not find '{config_file}'.")
    print('💡 Run 0_setup/1_setup.ipynb first to create it.')
    raise e

FOUNDRY_NAME = config.get('FOUNDRY_NAME')
RESOURCE_GROUP = config.get('RESOURCE_GROUP')
LOCATION = config.get('LOCATION')
TENANT_ID = config.get('TENANT_ID')
PROJECT_NAME = config.get('PROJECT_NAME', 'proj-default')
AZURE_AI_PROJECT_ENDPOINT = config.get('AZURE_AI_PROJECT_ENDPOINT')
FOUNDRY_API_KEY = config.get('FOUNDRY_API_KEY')
AZURE_SUBSCRIPTION_ID = config.get('AZURE_SUBSCRIPTION_ID')

os.environ['FOUNDRY_NAME'] = FOUNDRY_NAME or ''
os.environ['LOCATION'] = LOCATION or ''
os.environ['RESOURCE_GROUP'] = RESOURCE_GROUP or ''
os.environ['AZURE_SUBSCRIPTION_ID'] = AZURE_SUBSCRIPTION_ID or ''
os.environ['AZURE_AI_PROJECT_ENDPOINT'] = AZURE_AI_PROJECT_ENDPOINT or ''
os.environ['FOUNDRY_API_KEY'] = FOUNDRY_API_KEY or ''

print(f"✅ Loaded settings from '{config_file}'.")
print(f"\n📌 Foundry name: {FOUNDRY_NAME}")
print(f"📌 Resource group: {RESOURCE_GROUP}")
print(f"📌 Location: {LOCATION}")
print(f"📌 Azure AI Project endpoint: {AZURE_AI_PROJECT_ENDPOINT}")

# Fleet overview - list deployed agents
credential = DefaultAzureCredential()

✅ Loaded settings from '../0_setup/.foundry_config.json'.

📌 Foundry name: foundry-lhnjfq
📌 Resource group: msfoundry-rg
📌 Location: northcentralus
📌 Azure AI Project endpoint: https://foundry-lhnjfq.services.ai.azure.com/api/projects/msfoundry-prj1


# Hosted Agent Registration overview

From agent perspective, list **deployed agents** in the project. (The SDK does not expose every Portal dashboard field, but it is useful for automated checks.)

## Configure Azure Container Registry permissions

- In the Azure portal, go to your Foundry project resource.

- On the left pane, select Identity.

- Under System assigned, copy the Object (principal) ID value. This value is the managed identity that you'll assign the Azure Container Instances role to.

- Grant pull permissions by assigning the Container Registry Repository Reader role to your project's managed identity on the container registry. For detailed steps, see Azure Container Registry roles and permissions.

## Create account-level Capability Host

**What is a Capability Host?**

A Capability Host is the infrastructure environment that enables Hosted Agents to run on Microsoft Foundry's managed platform. It provides the compute, networking, and runtime resources needed to deploy and scale containerized agents.

**Why is it required?**

Before deploying any Hosted Agent, you must create an account-level Capability Host with `enablePublicHostingEnvironment: true`. This one-time setup:

- Provisions the hosting infrastructure in your Foundry account
- Enables container orchestration for agent deployments
- Configures networking and security boundaries for hosted workloads

**Note:** If you already have a Capability Host, you must delete it and recreate with the correct properties. Updating existing Capability Hosts is not supported.

In [4]:
import json
import subprocess
import os

subscription_id = os.environ.get("AZURE_SUBSCRIPTION_ID", "")
resource_group = RESOURCE_GROUP
foundry_name = FOUNDRY_NAME

if not subscription_id:
    raise RuntimeError("AZURE_SUBSCRIPTION_ID is not set.")

# Configuration: Set to True to create Capability Host
REMOVE_CAPHOST = False  # Change to False to skip creation
CREATE_CAPHOST = False  # Change to False to skip creation

# Capability Host URL
caphost_url = (
    f"https://management.azure.com/subscriptions/{subscription_id}"
    f"/resourceGroups/{resource_group}"
    f"/providers/Microsoft.CognitiveServices/accounts/{foundry_name}"
    f"/capabilityHosts/accountcaphost"
    "?api-version=2025-10-01-preview"
)

# Step 1: Delete existing Capability Host if requested
if REMOVE_CAPHOST:
    print(f"🗑️  Deleting existing Capability Host (if any): accountcaphost")
    print("   (Updating capability hosts is not supported, so we delete and recreate)\n")
    
    delete_result = subprocess.run(
        ["az", "rest", "--method", "DELETE", "--url", caphost_url],
        capture_output=True,
        text=True,
    )
    
    if delete_result.returncode == 0:
        print("✅ Existing Capability Host deleted (or none existed)")
    else:
        # 404 means it didn't exist, which is fine
        if "404" in delete_result.stderr or "NotFound" in delete_result.stderr:
            print("ℹ️  No existing Capability Host found (nothing to delete)")
        else:
            print(f"⚠️  Delete failed (continuing anyway): {delete_result.stderr[:200]}")
    
    print()

if CREATE_CAPHOST:
# Step 2: Create new Capability Host
    caphost_body = json.dumps(
        {
            "properties": {
                "capabilityHostKind": "Agents",
                "enablePublicHostingEnvironment": True
            }
        }
    )

    print(f"📌 Creating account-level Capability Host for: {foundry_name}")
    print("   capabilityHostKind: Agents")
    print("   enablePublicHostingEnvironment: True")
    print(f"   API version: 2025-10-01-preview\n")

    result = subprocess.run(
        ["az", "rest", "--method", "PUT", "--url", caphost_url, "--body", caphost_body],
        capture_output=True,
        text=True,
    )

    if result.returncode != 0:
        raise RuntimeError(f"Capability Host creation failed: {result.stderr}")

    print("✅ Capability Host created successfully")
    caphost_info = json.loads(result.stdout)
    caphost_id = caphost_info.get("id", "")
    print(f"   Capability Host ID: {caphost_id}")

    properties = caphost_info.get("properties", {})
    kind = properties.get("capabilityHostKind")
    enabled = properties.get("enablePublicHostingEnvironment")

    print(f"   capabilityHostKind: {kind}")
    print(f"   enablePublicHostingEnvironment: {enabled}")

## Register a hosted Agent via Azure Container Registry (Python SDK)
This section contains a minimal Python hosted agent packaged as a Docker container.
Microsoft Foundry hosted agents are deployed from container images stored in Azure Container Registry (ACR).

Reference: [What are hosted agents? (Build and push your Docker image to Azure Container Registry)](https://learn.microsoft.com/en-us/azure/ai-foundry/agents/concepts/hosted-agents?view=foundry&tabs=foundry-sdk#build-and-push-your-docker-image-to-azure-container-registry)

## Prerequisites
- Docker installed and running.
- Azure CLI (`az`) installed and signed in.
- An existing Azure Container Registry.

## Build and push an image to ACR Example

### 0) Update az cli (if needed)
```bash
az upgrade
```

### 1) Build the image locally
From this directory:

```bash
docker build -t hostedagent:1 .
```

### 2) Create an Azure Container Registry (if needed)
If you don't have an ACR, create one:

```bash
az acr create --name <myregistry> --resource-group <my-rg> --sku Basic
```

### 3) Sign in to Azure Container Registry
```bash
az acr login --name <myregistry>
```

### 4) Tag the image for your registry
```bash
docker tag hostedagent:1 <myregistry>.azurecr.io/hostedagent:1
```

### 5) Push the image
```bash
docker push <myregistry>.azurecr.io/hostedagent:1

## Case 1: MSFT Doc Agent Registration (MAF + MCPStreamableHTTPTool)

### Build Your Docker Image 

In [27]:
# Shared setup: Project endpoint / model deployment name
from datetime import datetime

AGENT_IMAGE_NAME = "msftdocagent:1"

print(f"AGENT_IMAGE_NAME = {AGENT_IMAGE_NAME}")

AZURE_CONTAINER_REGISTRY = os.environ.get("AZURE_CONTAINER_REGISTRY")

if not AZURE_CONTAINER_REGISTRY:
    print("WARNING: AZURE_CONTAINER_REGISTRY is empty.")
else:
    print(f"AZURE_CONTAINER_REGISTRY = {AZURE_CONTAINER_REGISTRY}")

AZURE_AI_PROJECT_ENDPOINT = os.environ.get("AZURE_AI_PROJECT_ENDPOINT")

if not AZURE_AI_PROJECT_ENDPOINT:
    print("WARNING: AZURE_AI_PROJECT_ENDPOINT is empty.")
else:
    print(f"AZURE_AI_PROJECT_ENDPOINT = {AZURE_AI_PROJECT_ENDPOINT}")

AZURE_AI_MODEL_DEPLOYMENT_NAME = os.environ.get("AZURE_AI_MODEL_DEPLOYMENT_NAME")

if not AZURE_AI_MODEL_DEPLOYMENT_NAME:
    print("WARNING: AZURE_AI_MODEL_DEPLOYMENT_NAME is empty.")
else:
    print(f"AZURE_AI_MODEL_DEPLOYMENT_NAME = {AZURE_AI_MODEL_DEPLOYMENT_NAME}")

AGENT_IMAGE_NAME = msftdocagent:1
AZURE_CONTAINER_REGISTRY = msfoundryacr.azurecr.io
AZURE_AI_PROJECT_ENDPOINT = https://foundry-lhnjfq.services.ai.azure.com/api/projects/msfoundry-prj1
AZURE_AI_MODEL_DEPLOYMENT_NAME = gpt-4.1


In [28]:
!az acr build --platform=linux/amd64 --image {AGENT_IMAGE_NAME} --registry {AZURE_CONTAINER_REGISTRY} --resource-group {RESOURCE_GROUP} --file ./1.1_hosted-agent_sdk/msft-docs-agent/Dockerfile ./1.1_hosted-agent_sdk/msft-docs-agent

Registry name is 'msfoundryacr'. The following suffix '.azurecr.io' is automatically omitted.
Packing source code into tar to upload...
Uploading archived source code from '/tmp/build_archive_79e9f573406e40368612409368331b3c.tar.gz'...
Sending context (1.430 KiB) to registry: msfoundryacr...
Queued a build with ID: cpb
Waiting for an agent...
2026/01/14 04:41:42 Downloading source code...
2026/01/14 04:41:43 Finished downloading source code
2026/01/14 04:41:43 Using acb_vol_56a1232e-7816-446d-8ee3-bbf242080411 as the home volume
2026/01/14 04:41:43 Setting up Docker configuration...
2026/01/14 04:41:44 Successfully set up Docker configuration
2026/01/14 04:41:44 Logging in to registry: msfoundryacr.azurecr.io
2026/01/14 04:41:44 Successfully logged into msfoundryacr.azurecr.io
2026/01/14 04:41:44 Executing step ID: build. Timeout(sec): 28800, Working directory: '', Network: ''
2026/01/14 04:41:44 Scanning for dependencies...
2026/01/14 04:41:45 Successfully scanned dependencies
2026/01

In [29]:
uuid_suffix = datetime.now().strftime("%Y%m%d%H%M%S")

AGENT_NAME=f"msft-docs-agent-{uuid_suffix}"
with (
    AIProjectClient(endpoint=AZURE_AI_PROJECT_ENDPOINT, credential=DefaultAzureCredential()) as project_client,
 ):
    agent = None
    conversation = None
    try:
        project_cs = project_client.telemetry.get_application_insights_connection_string()
        # Create the agent from a container image
        agent = project_client.agents.create_version(
            agent_name=AGENT_NAME,
            definition=ImageBasedHostedAgentDefinition(
                kind="hosted",
                container_protocol_versions=[ProtocolVersionRecord(protocol=AgentProtocol.RESPONSES, version="v1")], # Use RESPONSES protocol v1
                cpu="1",
                memory="2Gi",
                image=f"{AZURE_CONTAINER_REGISTRY}/{AGENT_IMAGE_NAME}",
                # Set environment variables, Checking the main.py and agent.yaml for your agent to match
                environment_variables={
                    "AZURE_AI_PROJECT_ENDPOINT": AZURE_AI_PROJECT_ENDPOINT,
                    "AZURE_AI_MODEL_DEPLOYMENT_NAME": AZURE_AI_MODEL_DEPLOYMENT_NAME,
                }
            )
        )
        
        print(f"{AGENT_NAME} created successfully.")
    
    except Exception as e:
        print(f"Error during agent creation: {repr(e)}")

msft-docs-agent-20260114044356 created successfully.


In [30]:
!az cognitiveservices agent start --account-name {FOUNDRY_NAME} --project-name {PROJECT_NAME} --name {AGENT_NAME} --agent-version 1

Command group 'cognitiveservices agent' is in preview and under development. Reference and support levels: https://aka.ms/CLI_refstatus
{
  "agent_id": "msft-docs-agent-20260114044356",
  "agent_version_id": "1",
  "container": {
    "created_at": "2026-01-14T04:44:00.9695183Z",
    "max_replicas": 1,
    "min_replicas": 1,
    "object": "agent.container",
    "status": "Starting",
    "updated_at": "2026-01-14T04:44:00.9695187Z"
  },
  "id": "44bb4e4e-c781-4b18-b60f-b4a8e974474c",
  "status": "InProgress"
}


## Case 2: Calculator Agent Registration (LangGraph + Hosting Adapter)

### Build Your Docker Image 

In [56]:
# Shared setup: Project endpoint / model deployment name
from datetime import datetime

AGENT_IMAGE_NAME = "calculatoragent:1"

print(f"AGENT_IMAGE_NAME = {AGENT_IMAGE_NAME}")

AZURE_CONTAINER_REGISTRY = os.environ.get("AZURE_CONTAINER_REGISTRY")

if not AZURE_CONTAINER_REGISTRY:
    print("WARNING: AZURE_CONTAINER_REGISTRY is empty.")
else:
    print(f"AZURE_CONTAINER_REGISTRY = {AZURE_CONTAINER_REGISTRY}")

AZURE_OPENAI_ENDPOINT = os.environ.get("AZURE_OPENAI_ENDPOINT")

if not AZURE_OPENAI_ENDPOINT:
    print("WARNING: AZURE_OPENAI_ENDPOINT is empty.")
else:
    print(f"AZURE_OPENAI_ENDPOINT = {AZURE_OPENAI_ENDPOINT}")

AZURE_AI_MODEL_DEPLOYMENT_NAME = os.environ.get("AZURE_AI_MODEL_DEPLOYMENT_NAME")

if not AZURE_AI_MODEL_DEPLOYMENT_NAME:
    print("WARNING: AZURE_AI_MODEL_DEPLOYMENT_NAME is empty.")
else:
    print(f"AZURE_AI_MODEL_DEPLOYMENT_NAME = {AZURE_AI_MODEL_DEPLOYMENT_NAME}")

AZURE_OPENAI_CHAT_DEPLOYMENT_NAME = os.environ.get("AZURE_OPENAI_CHAT_DEPLOYMENT_NAME")

if not AZURE_OPENAI_CHAT_DEPLOYMENT_NAME:
    print("WARNING: AZURE_OPENAI_CHAT_DEPLOYMENT_NAME is empty.")
else:
    print(f"AZURE_OPENAI_CHAT_DEPLOYMENT_NAME = {AZURE_OPENAI_CHAT_DEPLOYMENT_NAME}")

AZURE_OPENAI_API_KEY = os.environ.get("AZURE_OPENAI_API_KEY")

if not AZURE_OPENAI_API_KEY:
    print("WARNING: AZURE_OPENAI_API_KEY is empty.")
else:
    print("AZURE_OPENAI_API_KEY is set.")

AGENT_IMAGE_NAME = calculatoragent:1
AZURE_CONTAINER_REGISTRY = msfoundryacr.azurecr.io
AZURE_OPENAI_ENDPOINT = https://foundry-lhnjfq.cognitiveservices.azure.com/
AZURE_AI_MODEL_DEPLOYMENT_NAME = gpt-4.1
AZURE_OPENAI_CHAT_DEPLOYMENT_NAME = gpt-4.1
AZURE_OPENAI_API_KEY is set.


In [57]:
!az acr build --platform=linux/amd64 --image {AGENT_IMAGE_NAME} --registry {AZURE_CONTAINER_REGISTRY} --resource-group {RESOURCE_GROUP} --file ./1.1_hosted-agent_sdk/calculator-agent/Dockerfile ./1.1_hosted-agent_sdk/calculator-agent

Registry name is 'msfoundryacr'. The following suffix '.azurecr.io' is automatically omitted.
Packing source code into tar to upload...
Uploading archived source code from '/tmp/build_archive_7e75f4682f0846e0985170a7b59a2a4c.tar.gz'...
Sending context (2.561 KiB) to registry: msfoundryacr...
Queued a build with ID: cpe
Waiting for an agent...
2026/01/14 05:38:45 Downloading source code...
2026/01/14 05:38:46 Finished downloading source code
2026/01/14 05:38:46 Using acb_vol_8abe84ca-7080-49a9-9ea5-124c8e9ebe8d as the home volume
2026/01/14 05:38:46 Setting up Docker configuration...
2026/01/14 05:38:47 Successfully set up Docker configuration
2026/01/14 05:38:47 Logging in to registry: msfoundryacr.azurecr.io
2026/01/14 05:38:48 Successfully logged into msfoundryacr.azurecr.io
2026/01/14 05:38:48 Executing step ID: build. Timeout(sec): 28800, Working directory: '', Network: ''
2026/01/14 05:38:48 Scanning for dependencies...
2026/01/14 05:38:48 Successfully scanned dependencies
2026/01

In [58]:
uuid_suffix = datetime.now().strftime("%Y%m%d%H%M%S")

AGENT_NAME=f"calculator-agent-{uuid_suffix}"
with (
    AIProjectClient(endpoint=AZURE_AI_PROJECT_ENDPOINT, credential=DefaultAzureCredential()) as project_client,
 ):
    agent = None
    conversation = None
    try:
        project_cs = project_client.telemetry.get_application_insights_connection_string()
        # Create the agent from a container image
        agent = project_client.agents.create_version(
            agent_name=AGENT_NAME,
            definition=ImageBasedHostedAgentDefinition(
                kind="hosted",
                container_protocol_versions=[ProtocolVersionRecord(protocol=AgentProtocol.RESPONSES, version="v1")], # Use RESPONSES protocol v1
                cpu="1",
                memory="2Gi",
                image=f"{AZURE_CONTAINER_REGISTRY}/{AGENT_IMAGE_NAME}",
                # Set environment variables, Checking the main.py and agent.yaml for your agent to match
                environment_variables={
                    "APPLICATIONINSIGHTS_CONNECTION_STRING": project_cs,
                    "AZURE_OPENAI_ENDPOINT": AZURE_OPENAI_ENDPOINT,
                    "AZURE_AI_MODEL_DEPLOYMENT_NAME": AZURE_AI_MODEL_DEPLOYMENT_NAME,
                    "AZURE_OPENAI_API_VERSION": "2025-03-01-preview",
                }
            )
        )
        
        print(f"{AGENT_NAME} created successfully.")
    
    except Exception as e:
        print(f"Error during agent creation: {repr(e)}")

calculator-agent-20260114054050 created successfully.


In [59]:
!az cognitiveservices agent start --account-name {FOUNDRY_NAME} --project-name {PROJECT_NAME} --name {AGENT_NAME} --agent-version 1

Command group 'cognitiveservices agent' is in preview and under development. Reference and support levels: https://aka.ms/CLI_refstatus
{
  "agent_id": "calculator-agent-20260114054050",
  "agent_version_id": "1",
  "container": {
    "created_at": "2026-01-14T05:40:55.2315626Z",
    "max_replicas": 1,
    "min_replicas": 1,
    "object": "agent.container",
    "status": "Starting",
    "updated_at": "2026-01-14T05:40:55.2315631Z"
  },
  "id": "1005746e-2209-44ae-9c95-ed4e403e7818",
  "status": "InProgress"
}


## Case 3: Workflow Agent Registration (MAF + Hosting Adapter + Concurrent Workflow)

### Build Your Docker Image 

In [40]:
# Shared setup: Project endpoint / model deployment name
from datetime import datetime

AGENT_IMAGE_NAME = "workflowagent:1"

print(f"AGENT_IMAGE_NAME = {AGENT_IMAGE_NAME}")

AZURE_CONTAINER_REGISTRY = os.environ.get("AZURE_CONTAINER_REGISTRY")

if not AZURE_CONTAINER_REGISTRY:
    print("WARNING: AZURE_CONTAINER_REGISTRY is empty.")
else:
    print(f"AZURE_CONTAINER_REGISTRY = {AZURE_CONTAINER_REGISTRY}")

AZURE_OPENAI_ENDPOINT = os.environ.get("AZURE_OPENAI_ENDPOINT")

if not AZURE_OPENAI_ENDPOINT:
    print("WARNING: AZURE_OPENAI_ENDPOINT is empty.")
else:
    print(f"AZURE_OPENAI_ENDPOINT = {AZURE_OPENAI_ENDPOINT}")

AZURE_OPENAI_CHAT_DEPLOYMENT_NAME = os.environ.get("AZURE_OPENAI_CHAT_DEPLOYMENT_NAME")

if not AZURE_OPENAI_CHAT_DEPLOYMENT_NAME:
    print("WARNING: AZURE_OPENAI_CHAT_DEPLOYMENT_NAME is empty.")
else:
    print(f"AZURE_OPENAI_CHAT_DEPLOYMENT_NAME = {AZURE_OPENAI_CHAT_DEPLOYMENT_NAME}")

AGENT_IMAGE_NAME = workflowagent:1
AZURE_CONTAINER_REGISTRY = msfoundryacr.azurecr.io
AZURE_OPENAI_ENDPOINT = https://foundry-lhnjfq.cognitiveservices.azure.com/
AZURE_OPENAI_CHAT_DEPLOYMENT_NAME = gpt-4.1


In [36]:
!az acr build --platform=linux/amd64 --image {AGENT_IMAGE_NAME} --registry {AZURE_CONTAINER_REGISTRY} --resource-group {RESOURCE_GROUP} --file ./1.1_hosted-agent_sdk/workflow-agent/Dockerfile ./1.1_hosted-agent_sdk/workflow-agent

Registry name is 'msfoundryacr'. The following suffix '.azurecr.io' is automatically omitted.
Packing source code into tar to upload...
Uploading archived source code from '/tmp/build_archive_526f57c24c2242a48f24f5fc5dc6c37b.tar.gz'...
Sending context (1.919 KiB) to registry: msfoundryacr...
Queued a build with ID: cpd
Waiting for an agent...
2026/01/14 04:47:56 Downloading source code...
2026/01/14 04:47:57 Finished downloading source code
2026/01/14 04:47:57 Using acb_vol_246ed6d3-d238-4d1a-9b6e-baf9e5484524 as the home volume
2026/01/14 04:47:57 Setting up Docker configuration...
2026/01/14 04:47:58 Successfully set up Docker configuration
2026/01/14 04:47:58 Logging in to registry: msfoundryacr.azurecr.io
2026/01/14 04:47:58 Successfully logged into msfoundryacr.azurecr.io
2026/01/14 04:47:58 Executing step ID: build. Timeout(sec): 28800, Working directory: '', Network: ''
2026/01/14 04:47:58 Scanning for dependencies...
2026/01/14 04:47:59 Successfully scanned dependencies
2026/01

In [41]:
uuid_suffix = datetime.now().strftime("%Y%m%d%H%M%S")

AGENT_NAME=f"workflow-agent-{uuid_suffix}"
with (
    AIProjectClient(endpoint=AZURE_AI_PROJECT_ENDPOINT, credential=DefaultAzureCredential()) as project_client,
 ):
    agent = None
    conversation = None
    try:
        project_cs = project_client.telemetry.get_application_insights_connection_string()
        # Create the agent from a container image
        agent = project_client.agents.create_version(
            agent_name=AGENT_NAME,
            definition=ImageBasedHostedAgentDefinition(
                kind="hosted",
                container_protocol_versions=[ProtocolVersionRecord(protocol=AgentProtocol.RESPONSES, version="v1")], # Use RESPONSES protocol v1
                cpu="1",
                memory="2Gi",
                image=f"{AZURE_CONTAINER_REGISTRY}/{AGENT_IMAGE_NAME}",
                # Set environment variables, Checking the main.py and agent.yaml for your agent to match
                environment_variables={
                    "APPLICATIONINSIGHTS_CONNECTION_STRING": project_cs,
                    "AZURE_OPENAI_ENDPOINT": AZURE_OPENAI_ENDPOINT,
                    "AZURE_OPENAI_CHAT_DEPLOYMENT_NAME": AZURE_OPENAI_CHAT_DEPLOYMENT_NAME
                }
            )
        )
        
        print(f"{AGENT_NAME} created successfully.")
    
    except Exception as e:
        print(f"Error during agent creation: {repr(e)}")

workflow-agent-20260114045144 created successfully.


In [42]:
!az cognitiveservices agent start --account-name {FOUNDRY_NAME} --project-name {PROJECT_NAME} --name {AGENT_NAME} --agent-version 1

Command group 'cognitiveservices agent' is in preview and under development. Reference and support levels: https://aka.ms/CLI_refstatus
{
  "agent_id": "workflow-agent-20260114045144",
  "agent_version_id": "1",
  "container": {
    "created_at": "2026-01-14T04:51:48.7883623Z",
    "max_replicas": 1,
    "min_replicas": 1,
    "object": "agent.container",
    "status": "Starting",
    "updated_at": "2026-01-14T04:51:48.7883627Z"
  },
  "id": "9e808058-4088-4c26-ab2d-8a777ac516e8",
  "status": "InProgress"
}


## Case 4: Web Search Agent Registration (MAF + Hosting Adapter + Grouding with Bing Search Connection)

### Create and Connect Grounding with Bing Search

This is a **portal step** (one-time setup per Foundry resource). Creating an Grounding with Bing Search resource in Azure does **not** automatically connect it to your Foundry project.

### Steps (MS Foundry portal)

![Connect Grounding with Bing Search](../images/groundingwithbing.png)

1. Open the **Foundry portal**: https://ai.azure.com and navigate to your **project**.
2. In the top navigation, go to **Operate > Admin > <your project>**.
3. Select **Add connection**:
   - Choose **Grounding with Bing Search** and click **Continue**.
   - Select the Bing resource you created earlier, give the connection a name, then click **Connect**.
   - Use the **connection name** from your Connected Resources list 

### After you connect it (use it in code)

- Fetch the connection and capture its ID (you’ll pass this to the agent/tool):

  connection = project_client.connections.get(connection_name="<your-bing-connection-name>")  
  BING_GROUNDING_CONNECTION_ID = connection.id  
  print(f"BING_GROUNDING_CONNECTION_ID = {BING_GROUNDING_CONNECTION_ID}")

- Use `BING_GROUNDING_CONNECTION_ID` when constructing the Bing grounding tool (for example: `BingGroundingTool(connection_id=BING_GROUNDING_CONNECTION_ID)`), so the agent can access Bing Search.

### Build Your Docker Image 

In [19]:
# Shared setup: Project endpoint / model deployment name
from datetime import datetime

AGENT_IMAGE_NAME = "websearchagent:1"

print(f"AGENT_IMAGE_NAME = {AGENT_IMAGE_NAME}")

AZURE_CONTAINER_REGISTRY = os.environ.get("AZURE_CONTAINER_REGISTRY")

if not AZURE_CONTAINER_REGISTRY:
    print("WARNING: AZURE_CONTAINER_REGISTRY is empty.")
else:
    print(f"AZURE_CONTAINER_REGISTRY = {AZURE_CONTAINER_REGISTRY}")

AZURE_AI_PROJECT_ENDPOINT = os.environ.get("AZURE_AI_PROJECT_ENDPOINT")

if not AZURE_AI_PROJECT_ENDPOINT:
    print("WARNING: AZURE_AI_PROJECT_ENDPOINT is empty.")
else:
    print(f"AZURE_AI_PROJECT_ENDPOINT = {AZURE_AI_PROJECT_ENDPOINT}")

BING_GROUNDING_CONNECTION_NAME = os.environ.get("BING_GROUNDING_CONNECTION_NAME")

if not BING_GROUNDING_CONNECTION_NAME:
    print("WARNING: BING_GROUNDING_CONNECTION_NAME is empty.")
else:
    print(f"BING_GROUNDING_CONNECTION_NAME = {BING_GROUNDING_CONNECTION_NAME}")

AZURE_AI_MODEL_DEPLOYMENT_NAME = os.environ.get("AZURE_AI_MODEL_DEPLOYMENT_NAME")

if not AZURE_AI_MODEL_DEPLOYMENT_NAME:
    print("WARNING: AZURE_AI_MODEL_DEPLOYMENT_NAME is empty.")
else:
    print(f"AZURE_AI_MODEL_DEPLOYMENT_NAME = {AZURE_AI_MODEL_DEPLOYMENT_NAME}")

AGENT_IMAGE_NAME = websearchagent:1
AZURE_CONTAINER_REGISTRY = msfoundryacr.azurecr.io
AZURE_AI_PROJECT_ENDPOINT = https://foundry-lhnjfq.services.ai.azure.com/api/projects/msfoundry-prj1
BING_GROUNDING_CONNECTION_NAME = groundingbingsearchmh8xm5
AZURE_AI_MODEL_DEPLOYMENT_NAME = gpt-4.1


In [18]:
!az acr build --platform=linux/amd64 --image {AGENT_IMAGE_NAME} --registry {AZURE_CONTAINER_REGISTRY} --resource-group {RESOURCE_GROUP} --file ./1.1_hosted-agent_sdk/web-search-agent/Dockerfile ./1.1_hosted-agent_sdk/web-search-agent

Registry name is 'msfoundryacr'. The following suffix '.azurecr.io' is automatically omitted.
Packing source code into tar to upload...
Uploading archived source code from '/tmp/build_archive_54d89f8db8b0491096a664d81ee9ae8b.tar.gz'...
Sending context (1.623 KiB) to registry: msfoundryacr...
Queued a build with ID: cpa
Waiting for an agent...
2026/01/14 04:28:28 Downloading source code...
2026/01/14 04:28:29 Finished downloading source code
2026/01/14 04:28:29 Using acb_vol_39816e36-604d-46d3-9278-f7db133842c7 as the home volume
2026/01/14 04:28:29 Setting up Docker configuration...
2026/01/14 04:28:30 Successfully set up Docker configuration
2026/01/14 04:28:30 Logging in to registry: msfoundryacr.azurecr.io
2026/01/14 04:28:30 Successfully logged into msfoundryacr.azurecr.io
2026/01/14 04:28:30 Executing step ID: build. Timeout(sec): 28800, Working directory: '', Network: ''
2026/01/14 04:28:30 Scanning for dependencies...
2026/01/14 04:28:31 Successfully scanned dependencies
2026/01

In [ ]:
from azure.ai.projects.models import (
    BingGroundingAgentTool,
    BingGroundingSearchToolParameters,
    BingGroundingSearchConfiguration,
)

uuid_suffix = datetime.now().strftime("%Y%m%d%H%M%S")

AGENT_NAME=f"web-search-agent-{uuid_suffix}"
project_client = AIProjectClient(
    endpoint=AZURE_AI_PROJECT_ENDPOINT,
    credential=DefaultAzureCredential()
)

agent = None
conversation = None
try:
    connection = project_client.connections.get(name=BING_GROUNDING_CONNECTION_NAME)
    BING_GROUNDING_CONNECTION_ID = connection.id
    print(f"BING_GROUNDING_CONNECTION_ID = {BING_GROUNDING_CONNECTION_ID}")
        
    
    # Create the agent from a container image
    agent = project_client.agents.create_version(
        agent_name=AGENT_NAME,
        definition=ImageBasedHostedAgentDefinition(
            kind="hosted",
            container_protocol_versions=[ProtocolVersionRecord(protocol=AgentProtocol.RESPONSES, version="v1")], # Use RESPONSES protocol v1
            cpu="1",
            memory="2Gi",
            image=f"{AZURE_CONTAINER_REGISTRY}/{AGENT_IMAGE_NAME}",
            # https://learn.microsoft.com/en-us/python/api/azure-ai-projects/azure.ai.projects.models.tool?view=azure-python-preview
            tools=[
                BingGroundingAgentTool(
                        bing_grounding=BingGroundingSearchToolParameters(
                            search_configurations=[
                                BingGroundingSearchConfiguration(
                                    project_connection_id=BING_GROUNDING_CONNECTION_ID
                                )
                            ]
                        )
                    )
            ],
            # Set environment variables, Checking the main.py and agent.yaml for your agent to match
            environment_variables={
                "AZURE_AI_PROJECT_ENDPOINT": AZURE_AI_PROJECT_ENDPOINT,
                "AZURE_AI_MODEL_DEPLOYMENT_NAME": AZURE_AI_MODEL_DEPLOYMENT_NAME,
                "BING_GROUNDING_CONNECTION_ID": BING_GROUNDING_CONNECTION_ID
            }
        )
    )
    
    print(f"{AGENT_NAME} created successfully.")

except Exception as e:
    print(f"Error during agent creation: {repr(e)}")

BING_GROUNDING_CONNECTION_ID = /subscriptions/3d4d3dd0-79d4-40cf-a94e-b4154812c6ca/resourceGroups/msfoundry-rg/providers/Microsoft.CognitiveServices/accounts/foundry-lhnjfq/projects/msfoundry-prj1/connections/groundingbingsearchmh8xm5
web-search-agent-20260114053409 created successfully.


In [55]:
!az cognitiveservices agent start --account-name {FOUNDRY_NAME} --project-name {PROJECT_NAME} --name {AGENT_NAME} --agent-version 1

Command group 'cognitiveservices agent' is in preview and under development. Reference and support levels: https://aka.ms/CLI_refstatus
{
  "agent_id": "web-search-agent-20260114053409",
  "agent_version_id": "1",
  "container": {
    "created_at": "2026-01-14T05:34:50.4971665Z",
    "max_replicas": 1,
    "min_replicas": 1,
    "object": "agent.container",
    "status": "Starting",
    "updated_at": "2026-01-14T05:34:50.4971673Z"
  },
  "id": "e0076d17-15fb-4a1e-bbb9-06562d680bc8",
  "status": "InProgress"
}


# Invoke Hosted Agent

In [61]:
from azure.identity import DefaultAzureCredential
from azure.ai.projects import AIProjectClient
from azure.ai.projects.models import AgentReference

# Configuration
AGENT_NAME = "workflow-agent-20260114045144"  # Use the agent name created earlier
AGENT_VERSION = "1"  # Optional: specify version, or use latest

with (
    DefaultAzureCredential() as credential,
    AIProjectClient(endpoint=AZURE_AI_PROJECT_ENDPOINT, credential=credential) as project_client,
 ):
    agent = project_client.agents.get(agent_name=AGENT_NAME)

    # Get the OpenAI client and send a message
    openai_client = project_client.get_openai_client()
    response = openai_client.responses.create(
        input=[{"role": "user", "content": "Tell me what you can do?"}],
        extra_body={"agent": AgentReference(name=agent.name, version=AGENT_VERSION).as_dict()}
    )

print(f"Agent response: {response.output_text}")

Agent response: Of course! Please provide your prompt or the topic you'd like insights on.Of course! Please provide the product, service, or concept you want value propositions and target messaging for.Of course! Based on your prompt, I'll highlight relevant constraints, disclaimers, and policy concerns you should consider:

General Legal & Compliance Constraints:
- No Legal Advice: Any information provided should not be construed as formal legal advice between lawyer and client. For specific legal decisions, consult a licensed attorney.
- Privacy & Data Protection: If your prompt involves handling personal or sensitive data, you must comply with regulations (GDPR, CCPA, HIPAA, etc.) and avoid collecting or sharing information that could identify individuals without appropriate consent.
- Intellectual Property: Ensure materials, content, or references do not infringe on third-party copyrights, trademarks, or patents. Proper attribution and licensing checks are necessary.
- Regulatory R

## Check the tracing of agents


### Steps (MS Foundry portal)
1. Open **Foundry portal**: https://ai.azure.com and navigate to your **project**.
2. In the top navigation, select **Build > Agents > your agent name**.
![Agent Tracing](../images/agent_tracing.png)
3. select **Trace ID** to view traces and spans. 
![Agent Tracing Detail](../images/agent_tracing_detail.png)

## Additional resources

- [Azure AI Foundry Control Plane overview](https://learn.microsoft.com/en-us/azure/ai-foundry/control-plane/overview?view=foundry)
- [Monitoring across fleet](https://learn.microsoft.com/en-us/azure/ai-foundry/control-plane/monitoring-across-fleet?view=foundry)
- [Register a custom agent](https://learn.microsoft.com/en-us/azure/ai-foundry/control-plane/register-custom-agent?view=foundry)
- [Quickstart: create a guardrail policy](https://learn.microsoft.com/en-us/azure/ai-foundry/control-plane/quickstart-create-guardrail-policy?view=foundry)
- [Manage compliance and security](https://learn.microsoft.com/en-us/azure/ai-foundry/control-plane/how-to-manage-compliance-security?view=foundry)
- [Optimize cost and performance](https://learn.microsoft.com/en-us/azure/ai-foundry/control-plane/how-to-optimize-cost-performance?view=foundry)

# Wrap-up

In this notebook, you did a quick Control Plane-style check using the SDK.

- Good fit for SDK: listing agents/connections, scripting inventory checks
- Portal-first tasks: quota changes, compliance/security dashboards, admin controls

Suggested next steps:
1. Review Fleet/Quota dashboards in the Portal
2. Automate periodic inventory checks in CI
3. Configure governance controls (guardrails/compliance) for production